In [25]:
import requests
import os
import sys
import pandas as pd
import json

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from ff14_utils import *

In [28]:
if 'google.colab' in sys.modules:
    root_dir = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ff14')
    data_dir = os.path.join(root_dir, 'data')
else:
    root_dir = os.getcwd()
    data_dir = os.path.join(root_dir, 'data')

In [29]:
ITEM_CATEGORY_CONFIG_MAP = json.load(open(os.path.join(data_dir, 'item_category_config_map.json')))

In [30]:
target_worlds = get_target_worlds(5)

In [31]:
path = os.path.join(data_dir, 'Items_v3.csv')

df = pd.read_csv(path, header=0)

In [32]:
categories = ITEM_CATEGORY_CONFIG_MAP.keys()
print(list(categories))

['All', 'Pugilist Weapons', 'One-handed Swords', 'Greataxes', 'Bows', 'Polearms', 'One-handed Conjurer Arms', 'Two-handed Conjurer Arms', 'One-handed Thaumaturge Arms', 'Two-handed Thaumaturge Arms', 'Grimoires', 'Shields', "Carpenter's Primary Tool", "Carpenter's Secondary Tool", "Blacksmith's Primary Tool", "Blacksmith's Secondary Tool", "Armorer's Primary Tool", "Armorer's Secondary Tool", "Goldsmith's Primary Tool", "Goldsmith's Secondary Tool", "Leatherworker's Primary Tool", "Leatherworker's Secondary Tool", "Weaver's Primary Tool", "Weaver's Secondary Tool", "Alchemist's Primary Tool", "Alchemist's Secondary Tool", "Culinarian's Primary Tool", "Culinarian's Secondary Tool", "Miner's Primary Tool", "Miner's Secondary Tool", "Botanist's Primary Tool", "Botanist's Secondary Tool", "Fisher's Primary Tool", 'Fishing Tackle', 'Head Armor', 'Body Armor', 'Leg Armor', 'Hand Armor', 'Foot Armor', 'Waist Armor', 'Necklaces', 'Earrings', 'Bracelets', 'Rings', 'Potions', 'Ingredients', 'Foo

In [33]:
all_items = df['#'].to_list()

fishs = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Fish']]['#'].to_list()
potions = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Potions']]['#'].to_list()
foods = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Food']]['#'].to_list()
ingredients = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Ingredients']]['#'].to_list()
shards = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Crystals']]['#'].to_list()
furnishings = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Furnishings']]['#'].to_list()
materias = df[df['ItemUICategory'] == ITEM_CATEGORY_CONFIG_MAP['Materia']]['#'].to_list()

miscellaneous_categories = ['Metals', 'Lumber', 'Cloth', 'Leather', 'Bone', 'Alchemical Materials', 'Dyes', 'Parts']
miscellaneous = df[df['ItemUICategory'].isin([ITEM_CATEGORY_CONFIG_MAP[category] for category in miscellaneous_categories])]['#'].to_list()

In [34]:
target_items = all_items

In [35]:
seconds_in_a_day = 60 * 60 * 24
milliseconds_in_a_day = seconds_in_a_day * 1000

In [36]:
path = os.path.join(data_dir, 'item_metadata.csv')
df_metadata = pd.read_csv(path, header=0)

In [37]:
df_metadata['worldID'] = df_metadata['worldID'].astype(int)

In [38]:
# remove the items from the cristal category
df_metadata = df_metadata[~df_metadata['itemID'].isin(shards)]

In [39]:
# split the data between hq and nq
df_metadata_hq = df_metadata.copy()
df_metadata_hq['itemID'] = df_metadata_hq['itemID'].apply(lambda x: f'{x}_hq')
df_metadata_hq.drop(columns=['nqSaleVelocity', 'averagePriceNQ'], inplace=True)
df_metadata_hq.rename(columns={'hqSaleVelocity': 'saleVelocity', 'averagePriceHQ': 'averagePrice'}, inplace=True)

df_metadata_nq = df_metadata.copy()
df_metadata_nq['itemID'] = df_metadata_nq['itemID'].apply(lambda x: f'{x}_nq')
df_metadata_nq.drop(columns=['hqSaleVelocity', 'averagePriceHQ'], inplace=True)
df_metadata_nq.rename(columns={'nqSaleVelocity': 'saleVelocity', 'averagePriceNQ': 'averagePrice'}, inplace=True)

df_metadata_split = pd.concat([df_metadata_hq, df_metadata_nq])

In [40]:
df_metadata_split = df_metadata_split[df_metadata_split['saleVelocity'] > 0]

In [41]:
current_date = pd.to_datetime('today').date()
df_metadata_split['update_delay'] = [int((current_date - pd.to_datetime(x).date()).days) for x in df_metadata_split['lastUploadTime']]
df_metadata_split.drop(columns=['lastUploadTime'], inplace=True)
df_metadata_split = df_metadata_split[df_metadata_split['update_delay'] < 3]
df_metadata_split.drop(columns=['update_delay'], inplace=True)

In [42]:
df_metadata_split.sort_values(by='saleVelocity', ascending=False, inplace=True)

In [43]:
df_metadata_split.head()

,itemID,worldID,saleVelocity,averagePrice
80379,27796_nq,97,70,125.0
39538,31985_nq,80,70,86.2
39972,33916_nq,80,70,335.8
25484,31991_nq,71,70,475.2
39552,31999_nq,80,70,298.6


In [44]:
df_best = df_metadata_split.copy()
df_best = df_best.head(1000)
df_best.drop(columns=['saleVelocity'], inplace=True)

In [45]:
df_best.head()

,itemID,worldID,averagePrice
80379,27796_nq,97,125.0
39538,31985_nq,80,86.2
39972,33916_nq,80,335.8
25484,31991_nq,71,475.2
39552,31999_nq,80,298.6


In [46]:
target_keys = [
    'hq',
    'lastReviewTime',
    'pricePerUnit',
    'quantity',
    'materia',
    'onMannequin',
    'tax',
]

columns = [
    'itemID',
    'worldID',
    'pricePerUnit',
    'quantity',
    'hq',
    'materia',
    'onMannequin',
    'tax',
    'lastReviewTime',
    'averagePrice',
]

fields_str = ','.join([f'listings.{key}' for key in target_keys])

In [47]:
df_items = pd.DataFrame(columns=columns)

for i in range(len(df_best)):
    item_id = df_best.iloc[i]['itemID'].split('_')[0]
    item_q = df_best.iloc[i]['itemID'].split('_')[1]
    hq = 'true' if item_q == 'hq' else 'false'
    world = df_best.iloc[i]['worldID']
    price = df_best.iloc[i]['averagePrice']

    data = get_current_data(world, item_id, '', hq, '', fields_str)['listings']

    df_temp = pd.DataFrame(columns=columns)
    df_temp['itemID'] = [f'{item_id}_{item_q}'] * len(data)
    df_temp['worldID'] = [world] * len(data)
    df_temp['pricePerUnit'] = [x['pricePerUnit'] for x in data]
    df_temp['quantity'] = [x['quantity'] for x in data]
    df_temp['hq'] = [x['hq'] for x in data]
    df_temp['materia'] = [x['materia'] for x in data]
    df_temp['onMannequin'] = [x['onMannequin'] for x in data]
    df_temp['tax'] = [x['tax'] for x in data]
    df_temp['lastReviewTime'] = [x['lastReviewTime'] for x in data]
    df_temp['averagePrice'] = [price] * len(data)
    df_items = pd.concat([df_items, df_temp])

df_items = df_items.reset_index(drop=True)

/tmp/ipykernel_3290/4014266788.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_items = pd.concat([df_items, df_temp])


In [48]:
df_items.head()

,itemID,worldID,pricePerUnit,quantity,hq,materia,onMannequin,tax,lastReviewTime,averagePrice
0,27796_nq,97,99,99,False,[],False,490,1708913811,125.0
1,27796_nq,97,100,99,False,[],False,495,1708913992,125.0
2,27796_nq,97,100,99,False,[],False,495,1708913992,125.0
3,27796_nq,97,100,99,False,[],False,495,1708913992,125.0
4,27796_nq,97,100,2,False,[],False,10,1708940281,125.0


In [56]:
df_louisoix = df_items[df_items['worldID'] == 39]

In [59]:
df_louisoix['price_total'] = df_louisoix['pricePerUnit'] * df_louisoix['quantity'] + df_louisoix['tax']
df_louisoix['profit'] = df_louisoix['averagePrice'] - df_louisoix['price_total'] - df_louisoix['tax']

df_louisoix.drop(columns=['worldID', 'pricePerUnit', 'quantity', 'hq', 'materia', 'onMannequin', 'tax', 'lastReviewTime', 'averagePrice', 'price_total'], inplace=True)

df_louisoix.sort_values(by='profit', ascending=False, inplace=True)

/tmp/ipykernel_3290/708023729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_louisoix['price_total'] = df_louisoix['pricePerUnit'] * df_louisoix['quantity'] + df_louisoix['tax']
/tmp/ipykernel_3290/708023729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_louisoix['profit'] = df_louisoix['averagePrice'] - df_louisoix['price_total'] - df_louisoix['tax']
/tmp/ipykernel_3290/708023729.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

In [58]:
df_louisoix.head()

,itemID,worldID,pricePerUnit,quantity,hq,materia,onMannequin,tax,lastReviewTime,averagePrice,priceDiff
3338,32004_nq,39,1100,8,False,[],False,440,1708983441,1985.4,885.4
3337,32004_nq,39,1100,8,False,[],False,440,1708983441,1985.4,885.4
14044,38936_nq,39,999,99,False,[],False,4945,1708982468,1739.6,740.6
2968,37819_nq,39,300,99,False,[],False,1485,1708981469,1023.4,723.4
2969,37819_nq,39,300,99,False,[],False,1485,1708981469,1023.4,723.4


In [ ]:
# implement strategy : buy low and sell high
# buy low : pricePerUnit < averagePrice
# sell high : averagePrice > buyPrice

